# Analysis on the Energy consumption of different carbon capture methods

In [11]:
# TODO: IMPORTS IN JUPYTER STOPPED WORKING - SEE BELOW - WHY IS THIS THE CASE?
# check path
print(sys.path)

['C:\\Program Files\\JetBrains\\PyCharm 2022.2\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2022.2\\plugins\\python\\helpers\\pydev', 'C:\\Users\\2270577A\\PycharmProjects\\PhD_LCA_TEA\\analysis\\preliminary', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\python310.zip', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\DLLs', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\lib', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA', '', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\lib\\site-packages', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\lib\\site-packages\\win32', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\lib\\site-packages\\win32\\lib', 'C:\\Users\\2270577A\\Anaconda3\\envs\\PhD_LCA_TEA\\lib\\site-packages\\Pythonwin']


In [10]:
# Import required packages
import matplotlib.pyplot as plt
import numpy as np
from functions.general.utility import kJ_to_kWh, MJ_to_kWh
from configs import gaussian, triangular
from functions.MC import make_dist


ModuleNotFoundError: No module named 'functions'

In [ ]:
# Option A: Vacuum Pressure Swing Adsorption (VPSA) (Pre-Comb)
process_name_A = "Pre combustion VPSA"
recovery_rate_A = 87.3  # [%] on a syngas basis
recovery_rate_A = recovery_rate_A / 100 # decimals
electricity_consumption_A = np.array([0.986, 0.698])  # [MJ/ kg CO2] when including/excluding CO2 compression
heat_consumption_A = 0.51  # [MJ/ kg CO2]
source_A = "http://dx.doi.org/10.1016/j.ijggc.2015.01.008"


In [ ]:
# Option B: Amine-based Carbon Capture (Post-Comb)
process_name_B = "Post Combustion Amine Capture"
recovery_rate_B = 90  # [%] on a flue gas basis
recovery_rate_B = recovery_rate_B / 100 # decimals
electricity_consumption_B = np.array([0.324, 0.324, 0.324, 0.504, 0.492, 0.392, 0.402, 0.324, 0.325, 0.52, 1.02, 0.05])  # [MJ/ kg CO2]
heat_consumption_B = np.array([3.52, 3.11, 3.53, 2.62, 2.41, 2.76, 3.17, 3.1, 3.09, 2.22, 2.32, 3.5])  # [MJ/ kg CO2]
source_B = ["http://dx.doi.org/10.1016/j.ijggc.2013.03.002", "http://dx.doi.org/10.1016/j.ijggc.2015.01.008"]

# COMMENTS:
# - Ref 1 - relates to first 10 data points
# - Ref 2 - relates to the final 2 data points
# - recovery rate based on Ref 2
# - Electricity consumption calculated from Table 2 of Ref 1 for the first 10 data points



In [ ]:
# Option C: Vacuum Pressure Swing Adsorption (VPSA) (Post-Comb)
process_name_C = "Post combustion VPSA"
recovery_rate_C = np.array([90.9, 93.4, 97.9, 74.4, 80.7, 91.3, 80, 91.1, 80.6, 93.3])  # [%] on a flue gas basis
recovery_rate_C = recovery_rate_C / 100 # decimals
electricity_consumption_C = np.array([0.8229, 0.5284, 0.594, 0.7236, 0.8293, 0.756, 0.5132, 0.6457, 0.5244, 0.4981])  # [MJ/ kg CO2]
heat_consumption_C = 0  # [MJ/ kg CO2]
source_C = "http://dx.doi.org/10.1016/j.jece.2017.07.029"

# COMMENTS:
# - Data taken from Table 6


# Option A - Vacuum Pressure Swing Adsorption (VPSA) (Pre-Comb)

In [ ]:
# Create model
def carbon_capture_VPSA_pre_comb():

    # Define required data
    recovery_mean = 0.873  # as decimal on a syngas basis
    recovery_std = recovery_mean * 0.1  # use an estimated std of 10%

    electricity_consumption_mean = 0.986  # [MJ/ kg CO2]
    electricity_consumption_std = electricity_consumption_mean * 0.1 # [MJ/ kg CO2] use an estimated std of 10%

    heat_consumption_mean = 0.51  # [MJ/ kg CO2]
    heat_consumption_std = heat_consumption_mean * 0.1 # [MJ/ kg CO2] use an estimated std of 10%

    # Calculate outputs
    recovery_out = np.random.normal(recovery_mean, recovery_std)  # decimal
    electricity_out = np.random.normal(electricity_consumption_mean, electricity_consumption_std)  # [MJ/ kg CO2]
    heat_out = np.random.normal(heat_consumption_mean, heat_consumption_std)  # [MJ/ kg CO2]

    return {"Recovery": recovery_out, "Electricity consumption": electricity_out, "Heat consumption": heat_out}


In [ ]:
# Test model
carbon_capture_VPSA_pre_comb()

# Option B - Amine-based Carbon Capture (Post-Comb)

In [ ]:
# Look at trends in data
plt.scatter(np.arange(len(electricity_consumption_B)), electricity_consumption_B, label="Electricity consumption")
plt.scatter(np.arange(len(heat_consumption_B)), heat_consumption_B, label="Heat consumption")
total_consumption_B = electricity_consumption_B + heat_consumption_B
plt.scatter(np.arange(len(heat_consumption_B)), total_consumption_B, label="Sum")
plt.xlabel("Samples")
plt.ylabel("Energy consumption [MJ/ kg CO2 removed]")
plt.legend()
plt.show()

A somewhat inverse relationship can be identified between electricity and heat consumption. The last two points from a different study don't quite fit in as well as the rest of the data.


In [ ]:
# Show fractions of electricity consumption related to complete consumption
electricity_fraction_B = electricity_consumption_B/total_consumption_B
print(electricity_fraction_B)

In [ ]:
# Discard outliers
electricity_fraction_B_clean = electricity_fraction_B[0:-2]
print("Array after discarding outliers:")
print(electricity_fraction_B_clean)

# Calculate mean ans std
electricity_fraction_B_mean = np.mean(electricity_fraction_B_clean)
electricity_fraction_B_std = np.std(electricity_fraction_B_clean)

print("Fraction of electricity consumption compared to total energy consumption:")
print("Mean:", electricity_fraction_B_mean)
print("Std:", electricity_fraction_B_std)


In [ ]:
# Calculate mean and std of total consumption
total_consumption_B_mean = np.mean(total_consumption_B)
total_consumption_B_std = np.std(total_consumption_B)
print("Total energy consumption:")
print("Mean:", total_consumption_B_mean)
print("Std:", total_consumption_B_std)


In [ ]:
# Create model
def carbon_capture_amine_post_comb():

    # Define required data
    recovery_mean = 0.90  # as decimal on a flue gas basis
    recovery_std = recovery_mean * 0.1  # use an estimated std of 10%

    total_consumption_mean = 3.36  # [MJ/ kg CO2]
    total_consumption_std = 0.32  # [MJ/ kg CO2]
    total_consumption = np.random.normal(total_consumption_mean, total_consumption_std)

    electricity_fraction = np.random.normal(0.121, 0.037)
    heat_fraction = 1-electricity_fraction

    # Calculate outputs
    recovery_out = np.random.normal(recovery_mean, recovery_std)  # decimal
    electricity_out = total_consumption * electricity_fraction  # [MJ/ kg CO2]
    heat_out = total_consumption * heat_fraction  # [MJ/ kg CO2]

    return {"Recovery": recovery_out, "Electricity consumption": electricity_out, "Heat consumption": heat_out}


In [ ]:
# Test final function
carbon_capture_amine_post_comb()

# Option C - Vacuum Pressure Swing Adsorption (VPSA) (Post-Comb)


In [ ]:
# Calculate mean and std
electricity_consumption_C_mean = np.mean(electricity_consumption_C)
electricity_consumption_C_std = np.std(electricity_consumption_C)
recovery_rate_C_mean = np.mean(recovery_rate_C)
recovery_rate_C_std = np.std(recovery_rate_C)

print("Electricity consumption:")
print("Mean:", electricity_consumption_C_mean)
print("Std:", electricity_consumption_C_std)

print("Recovery rate:")
print("Mean:", recovery_rate_C_mean)
print("Std:", recovery_rate_C_std)

In [ ]:
# Show electricity consumption distribution
plt.scatter(np.arange(len(electricity_consumption_C)), np.sort(electricity_consumption_C))
plt.xlabel("Samples")
plt.ylabel("Energy consumption [MJ/ kg CO2 removed]")
plt.show()

In [ ]:
# Show recovery distribution
plt.scatter(np.arange(len(recovery_rate_C)), np.sort(recovery_rate_C))
plt.xlabel("Samples")
plt.ylabel("Recovery")
plt.show()

In [ ]:
# Create model
def carbon_capture_VPSA_post_comb():

    # Define required data
    recovery_mean = 0.874  # as decimal on a flue gas basis
    recovery_std = 0.073

    electricity_consumption_mean = 0.644  # [MJ/ kg CO2]
    electricity_consumption_std = 0.124  # [MJ/ kg CO2]
    electricity_consumption = np.random.normal(electricity_consumption_mean, electricity_consumption_std)

    heat_consumption = 0  # no heat required

    # Calculate outputs
    recovery_out = np.random.normal(recovery_mean, recovery_std)  # decimal
    electricity_out = np.random.normal(electricity_consumption_mean, electricity_consumption_std)  # [MJ/ kg CO2]
    heat_out = heat_consumption  # [MJ/ kg CO2]

    return {"Recovery": recovery_out, "Electricity consumption": electricity_out, "Heat consumption": heat_out}


In [ ]:
# Test model
carbon_capture_VPSA_post_comb()

# Sensibility check - using data from Option C

In [ ]:

energy_consumptions = MJ_to_kWh(electricity_consumption_C) # [kWh/kg CO2]
recovery = recovery_rate_C

# Calculate mean and std
energy_consumption_mean = np.mean(energy_consumptions)
energy_consumption_std = np.std(energy_consumptions)
recovery_mean = np.mean(recovery)
recovery_std = np.std(recovery)


In [ ]:
# Create distribution to generate histograms
energy_consumption_distribution = make_dist(gaussian(energy_consumption_mean, energy_consumption_std))
recovery_distribution = make_dist(triangular(lower=min(recovery), mode=recovery_mean, upper=max(recovery)))


In [ ]:
print("energy consumption")
plt.figure()
histogram = plt.hist(energy_consumption_distribution,20)
plt.xlabel("Energy consumption [kWh/kg CO2]")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print("Gas carbon recovery")
plt.figure()
histogram = plt.hist(recovery_distribution,20)
plt.xlabel("Carbon recovery")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Sensibility check of data
from processes.syngas_combustion import syngas_combustion_GWP_MC
from functions.LCA import electricity_GWP

# Calculate GWPs of syngas combustion
GWP_results = syngas_combustion_GWP_MC()

# Replace GWP results with approximated average GWP from biogenic and non-biogenic sources
GWP_simplified = 1800  # kg CO2/FU

neg_emissions = - GWP_simplified * recovery_mean  # kg CO2/FU
energy_req_emissions = electricity_GWP(energy_consumption_mean * GWP_simplified)

# Get current approximate avoided emissions due to electricity substitution - as used in the model right now.
GWP_avoided_elect_emissions= 180  # kg CO2/FU

In [ ]:
print("Avoided emissions from incorporating carbon capture:", neg_emissions, "kg CO2/ FU")
print("Emission penalty due to increased energy consumption:", energy_req_emissions, "kg CO2/ FU")
print("Percentage emission from additional energy requirements vs avoided emissions due to capture:",-energy_req_emissions/neg_emissions * 100, "%")
print("Percentage emission from additional energy requirements vs previous avoided emissions:",energy_req_emissions/GWP_avoided_elect_emissions * 100, "%")


# Summary
- Results are promising.
- Looks like avoided emission from electricity substitution would be decreased by 40% - i.e. 40% less electricity available for export.
- Emissions due to increased electricity usage are 5% of emissions that could be avoided due to CC.